In [ ]:
import pygame
import math
import random
import time
import numpy as np
from scipy.optimize import minimize

pygame.init()

# SCREEN DIMENSIONS
w, h = 800, 600

# SPEEDS
FPS = 120
evader_speed = 6.0
pursuer_speed = 2.0

# CBF SETTINGS
CBF_ALPHA = 0.9 # CBF gain 'alpha' (h_dot >= -alpha * h)

#GOALS
evader_radius, pursuer_radius, goal_radius = 15, 15, 20
max_goals = 10
MIN_GOAL_DIST_FROM_PURSUER = 100  # Minimum distance goal must be from pursuer

# Collision radius: actual physical collision (sum of agent radii)
COLLISION_RADIUS = evader_radius + pursuer_radius  # = 30 (actual collision)

#safety radius
CBF_SAFETY_RADIUS = 75.0  # Larger than collision radius for safety margin

# COLORS
WHITE = (255,255,255)
BLUE = (50,150,255)
RED = (255,80,80)
GREEN = (50,200,50)
BLACK = (0,0,0)

# initial state
evader_x = w//4
evader_y = h//2
pursuer_x = 3*w//4
pursuer_y = h//2
pursuer_vx = 0.0  # Track pursuer velocity for CBF
pursuer_vy = 0.0
goal_counter = 0
goal_times = []

# SCREEN
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("True CBF Implementation")
clock = pygame.time.Clock()

#SPAWN GOALS STRATEGICALLY
def spawn_goal_away_from_pursuer(pux, puy):
    max_attempts = 100
    for _ in range(max_attempts):
        gx = random.randint(0, w)
        gy = random.randint(0, h)
        dist = math.hypot(gx - pux, gy - puy)
        if dist >= MIN_GOAL_DIST_FROM_PURSUER:
            return gx, gy
    # Fallback: if we can't find a good spot after many attempts, return a position anyway
    return random.randint(0, w), random.randint(0, h)

# Spawn the initial goal
goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer_x, pursuer_y)
goal_spawn_time = pygame.time.get_ticks() / 1000.0

# Start the game
running = True
game_over = False
frame_count = 0
cbf_is_active = False  # State variable to track when CBF modifies velocity

########################################################
# get h(x)  - CBF fucntion #1
########################################################
def get_barrier_value(evx, evy, pux, puy, safety_radius=None):
    if safety_radius is None:
        safety_radius = CBF_SAFETY_RADIUS
    rx = evx - pux
    ry = evy - puy
    dist_sq = rx*rx + ry*ry
    return dist_sq - safety_radius**2

########################################################
# solve QP - CBF function #2
########################################################
def cbf_qp(evx, evy, pux, puy, pvx, pvy, desired_vx, desired_vy,
           safety_radius=None, alpha=None, max_speed=None):
    # Defaults
    if safety_radius is None:
        safety_radius = CBF_SAFETY_RADIUS #set safety radius
    if alpha is None:
        alpha = CBF_ALPHA #set alpha
    if max_speed is None:
        max_speed = evader_speed #set max speed
    
    # Relative position: r = x_e - x_p
    rx = evx - pux #distance between evader and pursuer 
    ry = evy - puy
    dist_sq = rx*rx + ry*ry #distance squared
    dist = math.sqrt(dist_sq) if dist_sq > 1e-6 else 1e-6 #distance
    
    # Barrier function: h = ||r||^2 - r^2
    h = dist_sq - safety_radius**2
    
    # Gradient: gradh = 2r
    grad_h_x = 2 * rx #gradient of h with respect to x
    grad_h_y = 2 * ry #gradient of h with respect to y
    
    # CBF constraint: grad h(v_e - v_p) >= -alpha h
    rel_vx_des = desired_vx - pvx #desired velocity - pursuer velocity
    rel_vy_des = desired_vy - pvy #desired velocity - pursuer velocity
    h_dot_desired = grad_h_x * rel_vx_des + grad_h_y * rel_vy_des #h dot desired
    desired_norm = math.hypot(desired_vx, desired_vy) #desired velocity norm
    
    # Check if desired velocity already satisfies CBF condition
    if desired_norm <= max_speed and h_dot_desired >= -alpha * h - 1e-6:
        return desired_vx, desired_vy, False
    
    # set up QP
    rhs = -alpha * h + grad_h_x * pvx + grad_h_y * pvy #right hand side of the QUADRATIC problem
    
    def objective(v):
        return (v[0] - desired_vx)**2 + (v[1] - desired_vy)**2 #objective function
    
    constraints = [{
        'type': 'ineq', #inequality constraint
        'fun': lambda v: grad_h_x * v[0] + grad_h_y * v[1] - rhs #constraint function
    }]
    
    if max_speed < float('inf'): #if the max speed is not infinite
        constraints.append({
            'type': 'ineq', #inequality constraint
            'fun': lambda v: max_speed**2 - (v[0]**2 + v[1]**2) #constraint function
        })
    
    # Initial guess: clamp desired to max_speed
    if desired_norm > max_speed:
        v0 = [desired_vx * max_speed / desired_norm, desired_vy * max_speed / desired_norm]
    else:
        v0 = [desired_vx, desired_vy]
    
    # Solve QP
    result = minimize(objective, v0, method='SLSQP', constraints=constraints,
                     options={'ftol': 1e-9, 'maxiter': 100, 'disp': False})
    
    #if the result is successful    
    if result.success:
        safe_vx, safe_vy = result.x[0], result.x[1] #safe velocity
        # Verify constraint is satisfied
        rel_vx = safe_vx - pvx #relative velocity
        rel_vy = safe_vy - pvy
        h_dot = grad_h_x * rel_vx + grad_h_y * rel_vy #h dot
        if h_dot >= -alpha * h - 1e-5:
            return safe_vx, safe_vy, True
    
    # Fallback: analytical projection onto constraint boundary
    grad_norm_sq = grad_h_x**2 + grad_h_y**2
    if grad_norm_sq > 1e-6:
        lambda_val = (rhs - grad_h_x * desired_vx - grad_h_y * desired_vy) / grad_norm_sq
        lambda_val = max(0.0, lambda_val)  # Only push outward
        safe_vx = desired_vx + lambda_val * grad_h_x
        safe_vy = desired_vy + lambda_val * grad_h_y
        # Clamp to max_speed
        safe_norm = math.hypot(safe_vx, safe_vy)
        if safe_norm > max_speed:
            safe_vx = safe_vx * max_speed / safe_norm
            safe_vy = safe_vy * max_speed / safe_norm
    else:
        # Degenerate: move directly away
        nx, ny = rx / dist, ry / dist
        safe_vx, safe_vy = max_speed * nx, max_speed * ny
    
    #return safe velocity 
    return safe_vx, safe_vy, True


# main loop
while running:

    #start the game
    clock.tick(FPS)
    screen.fill(WHITE)
    frame_count += 1

    # handle quit event
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            game_over = True

    #otherwise if not quit:
    if not game_over:
        # Goal-seeking desired velocity (task objective ONLY)
        gdx = goal_x - evader_x
        gdy = goal_y - evader_y
        gdist = math.hypot(gdx, gdy)
        
        if gdist > 1e-6:
            desired_vx = (gdx / gdist) * evader_speed
            desired_vy = (gdy / gdist) * evader_speed
        else:
            desired_vx = desired_vy = 0.0

        # Apply CBF QP filter (ONLY safety)
        safe_vx, safe_vy, cbf_is_active = cbf_qp(
            evader_x, evader_y, pursuer_x, pursuer_y,
            pursuer_vx, pursuer_vy,
            desired_vx, desired_vy
        )

        # Mode for visualization only - CBF is always active and enforces safety - CBF NOT A SWITCHING FILTER LOL
        h_value = get_barrier_value(evader_x, evader_y, pursuer_x, pursuer_y)
        if h_value < 0 or cbf_is_active:
            mode = "SAFETY"
        else:
            mode = "GOAL"

        # update the evader position using CBF-filtered velocity
        evader_x += safe_vx
        evader_y += safe_vy

        # pursuer pure pursuit calc
        pdx = evader_x - pursuer_x # distance between evader and pursuer
        pdy = evader_y - pursuer_y
        pdist = math.hypot(pdx, pdy) # distance between evader and pursuer
        if pdist > 1e-6: # if the distance is greater than 0, the pursuer is moving
            # Calculate pursuer velocity for CBF
            pursuer_vx = (pdx / pdist) * pursuer_speed
            pursuer_vy = (pdy / pdist) * pursuer_speed
            pursuer_x += pursuer_vx
            pursuer_y += pursuer_vy
        else:
            pursuer_vx, pursuer_vy = 0.0, 0.0
        pursuer_x = max(pursuer_radius, min(w - pursuer_radius, pursuer_x)) # keep pursuer in bounds
        pursuer_y = max(pursuer_radius, min(h - pursuer_radius, pursuer_y))
        evader_x = max(evader_radius, min(w - evader_radius, evader_x)) # keep evader in bounds
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        # Collision check: use actual collision radius (not CBF safety radius)
        # Collision occurs when distance < sum of agent radii
        dist_sq = (evader_x - pursuer_x)**2 + (evader_y - pursuer_y)**2
        if dist_sq < COLLISION_RADIUS**2:  # Actual physical collision
            game_over = True
            running = False
        if goal_counter < max_goals and math.hypot(evader_x - goal_x, evader_y - goal_y) <= evader_radius + goal_radius: # if the evader is close to the goal
            goal_times.append(pygame.time.get_ticks() / 1000.0 - goal_spawn_time) # add the time it took to reach the goal
            goal_counter += 1 # increment the goal counter
            if goal_counter < max_goals: # if the goal counter is less than the max goals   
                goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer_x, pursuer_y) # spawn a new goal
                goal_spawn_time = pygame.time.get_ticks() / 1000.0 # update the goal spawn time 
            else:
                game_over = True # game over
                running = False

    ### #################################
    ########## everything below this is VISUALIZATION ##########
    ####################################
    # Draw CBF safety radius (larger, light blue)
    pygame.draw.circle(screen, (200, 200, 255), (int(pursuer_x), int(pursuer_y)), int(CBF_SAFETY_RADIUS), 2)
    # Draw collision radius (smaller, red dashed - actual collision boundary)
    pygame.draw.circle(screen, (255, 100, 100), (int(pursuer_x), int(pursuer_y)), int(COLLISION_RADIUS), 1)

    # draw the pursuer, evader, and goal
    pygame.draw.circle(screen, RED, (int(pursuer_x), int(pursuer_y)), pursuer_radius)
    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)
    pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(pursuer_x), int(pursuer_y)), 1)

    # draw the text
    font = pygame.font.Font(None, 24)
    small_font = pygame.font.Font(None, 20)
    if not game_over:
        screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 8))
        mode_text = f"Mode: {mode}"
        if cbf_is_active:
            mode_text += " [CBF ACTIVE]"
        screen.blit(font.render(mode_text, True, RED if mode!="GOAL" else GREEN), (10, 32))
        screen.blit(small_font.render(f"CBF Safety R: {CBF_SAFETY_RADIUS:.1f}", True, (100, 100, 200)), (10, 56))
        screen.blit(small_font.render(f"Collision R: {COLLISION_RADIUS:.1f}", True, (200, 100, 100)), (10, 76))
        # Display barrier function value (h >= 0 is safe)
        h_val = get_barrier_value(evader_x, evader_y, pursuer_x, pursuer_y)
        h_color = GREEN if h_val >= 0 else RED
        screen.blit(small_font.render(f"h(x): {h_val:.1f}", True, h_color), (10, 96))
    else:
        if goal_counter >= max_goals:
            msg = font.render(f"VICTORY! {goal_counter} goals", True, GREEN)
        else:
            msg = font.render("CAUGHT - game over", True, RED)
        screen.blit(msg, msg.get_rect(center=(w//2, h//2)))

    # update the screen
    pygame.display.flip()

pygame.quit()

